In [1]:
# Imports
import sys
sys.path.append("..")
import os
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.autograd import Variable

# Local Imports
from src.models.CNN import CNNModel
from src.data.voxelize import voxelize
from src.data.utils import read_voxel_data

In [2]:
# Paths
LABELED_DATA = os.path.abspath("../data/labeled/")
VOXELIZED_DATA = os.path.abspath("../data/voxelized/amazon.h5py")

# Model Variables 
BATCH_SIZE = 3
NUM_CLASSES = 2
NUM_EPOCHS = 1
LEARNING_RATE = 0.001

In [3]:
# Voxelize the data
#voxelized_data_file = voxelize(LABELED_DATA, VOXELIZED_DATA, 100) # This takes a while

Voxel Size: 100
Voxelized: labeled_tile_-60_-30.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-30_-10.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-60_-20.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_0_0.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-40_-10.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-50_-10.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-50_-20.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-40_-20.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-10_-10.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-40_-30.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_0_10.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-60_-10.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-30_-30.txt | Shape: (100, 100, 400, 8)
Voxelized: labeled_tile_-30_-20.txt | Shape: (100, 100, 400, 8)
Output: /Users/julius/Desktop/LeafAndWood/data/voxelized/amazon.h5py


In [3]:
# Read voxelized data
voxels, labels = read_voxel_data(VOXELIZED_DATA)
train_x = torch.from_numpy(voxels).float()
train_y = torch.from_numpy(labels).long()
train = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)

Voxels data shape: (14, 100, 100, 400, 7)
Labels data shape: (14, 100, 100, 400)


In [4]:
# Model
model = CNNModel()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
print(model)

CNNModel(
  (conv_block_1): Sequential(
    (0): Conv3d(7, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (linear_block): Sequential(
    (0): Linear(in_features=3317888, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.15, inplace=False)
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)


In [5]:
# Training 
for epoch in range(NUM_EPOCHS):
    for i, (points, labels) in enumerate(train_loader):
        train = Variable(points.view(BATCH_SIZE, 7, 100, 100, 400))
        labels = Variable(labels.view(BATCH_SIZE, 1, 100, 100, 400))
        optimizer.zero_grad()
        outputs = model(train)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported